In [5]:
from google.colab import files

uploaded = files.upload()

Saving doencas_respiratorias_cronicas.pdf to doencas_respiratorias_cronicas.pdf
Saving dicionario_de_dados.docx to dicionario_de_dados.docx


In [1]:
# 📍 Instalar as dependências necessárias
%pip install transformers sentencepiece python-docx pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 59.3 MB/s eta 0:00:00


In [2]:
# 📚 Imports
import fitz  # pymupdf
import docx
from transformers import pipeline
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# 📂 Função para carregar e extrair texto de arquivos
def extract_text_from_docx(path):
    import docx
    doc = docx.Document(path)
    full_text = []

    # Extrair parágrafos comuns
    for p in doc.paragraphs:
        if p.text.strip():
            full_text.append(p.text.strip())

    # Extrair conteúdo das tabelas
    for table in doc.tables:
        for row in table.rows:
            row_text = [cell.text.strip() for cell in row.cells if cell.text.strip()]
            if row_text:
                full_text.append(" | ".join(row_text))

    return "\n".join(full_text)

def extract_text_from_pdf(path):
    doc = fitz.open(path)
    return "\n".join([page.get_text() for page in doc])

In [6]:
# 📄 Carregando arquivos (substitua os paths pelos seus uploads)
docx_path = "/content/dicionario_de_dados.docx"
pdf_path = "/content/doencas_respiratorias_cronicas.pdf"

# 📄 Extraindo conteúdo dos arquivos
docx_text = extract_text_from_docx(docx_path)
pdf_text = extract_text_from_pdf(pdf_path)

In [7]:
# Modelos Hugging Face
model_names = [
    "deepset/roberta-base-squad2",
    "distilbert-base-cased-distilled-squad",
    "ktrapeznikov/albert-xlarge-v2-squad-v2"
]
qa_pipelines = {
    name: pipeline("question-answering", model=name, tokenizer=name)
    for name in model_names
}

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/235M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
# Separar perguntas e respostas por fonte
docx_questions = [
    "Na tabela FCESGEST, quais os domínios do campo INDTIPOCAD?",
    "Na tabela LFCES004, qual a descrição do campo PFPJ_IND",
    "Na tabela LFCES035, qual nme do campo responsável por armazenar a Data do Processamento e seu tipo?"
]

pdf_questions = [
    "Com quais doenças respiratórias o baqueteamento digital pode estar associado?",
    "Quais são os principais sintomas da conjuntivite e rinite alérgica?",
    "Qual é o papel da lavagem nasal com solução salina no tratamento da rinite alérgica?"
]

respostas_esperadas = {
    # PDF
    pdf_questions[0]: "Doenças cardiovasculares, digestivas e pulmonares.",
    pdf_questions[1]: "Prurido, lacrimejamento, sensação de corpo estranho no olho, fotofobia, hiperemia e edema periocular.",
    pdf_questions[2]: "Aliviar a irritação tecidual, umedecer a mucosa e auxiliar na remoção de secreções, aliviando temporariamente a obstrução nasal e melhorando o olfato.",
    # DOCX
    docx_questions[0]: "1 - Completo\n2 - Só Profissionais",
    docx_questions[1]: "Indica se é Pessoa Física ou Jurídica",
    docx_questions[2]: "DT_PROCESS - DATE",
}

In [9]:
def calcular_similaridade(a, b):
    vect = TfidfVectorizer().fit([a, b])
    tfidf = vect.transform([a, b])
    return round(cosine_similarity(tfidf[0:1], tfidf[1:2])[0][0], 3)

In [10]:
def ask_questions_with_comparison(text, questions, source_name, respostas_esperadas):
    resultados = []

    for pergunta in questions:
        resposta_esperada = respostas_esperadas.get(pergunta, "")

        for model_name, qa in qa_pipelines.items():
            try:
                output = qa(question=pergunta, context=text)
                resposta_modelo = output.get("answer", "")
                score = round(output.get("score", 0.0), 3)
                similaridade = calcular_similaridade(resposta_modelo, resposta_esperada)

                resultados.append({
                    "Fonte": source_name,
                    "Pergunta": pergunta,
                    "Modelo": model_name,
                    "Resposta do Modelo": resposta_modelo,
                    "Score do Modelo": score,
                    "Resposta Esperada": resposta_esperada,
                    "Similaridade com Esperada": similaridade
                })

            except Exception as e:
                resultados.append({
                    "Fonte": source_name,
                    "Pergunta": pergunta,
                    "Modelo": model_name,
                    "Resposta do Modelo": f"Erro: {str(e)}",
                    "Score do Modelo": 0.0,
                    "Resposta Esperada": resposta_esperada,
                    "Similaridade com Esperada": 0.0
                })

    return resultados

In [11]:
# ✅ Executar QA para ambos os documentos
resultados_docx = ask_questions_with_comparison(docx_text, docx_questions, "Dicionário de Dados", respostas_esperadas)
resultados_pdf = ask_questions_with_comparison(pdf_text, pdf_questions, "Doenças Respiratórias", respostas_esperadas)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [16]:
# 📊 Juntar os resultados e exibir em uma tabela
df_resultado = pd.DataFrame(resultados_docx + resultados_pdf)
pd.set_option('display.max_colwidth', None)

In [17]:

# Mostrar os resultados ordenados por pergunta
df_resultado.sort_values(by=["Fonte", "Pergunta", "Modelo"], inplace=True)
df_resultado

,Fonte,Pergunta,Modelo,Resposta do Modelo,Score do Modelo,Resposta Esperada,Similaridade com Esperada
0,Dicionário de Dados,"Na tabela FCESGEST, quais os domínios do campo INDTIPOCAD?",deepset/roberta-base-squad2,\n,0.015,1 - Completo\n2 - Só Profissionais,0.000
1,Dicionário de Dados,"Na tabela FCESGEST, quais os domínios do campo INDTIPOCAD?",distilbert-base-cased-distilled-squad,NFCES015,0.005,1 - Completo\n2 - Só Profissionais,0.000
2,Dicionário de Dados,"Na tabela FCESGEST, quais os domínios do campo INDTIPOCAD?",ktrapeznikov/albert-xlarge-v2-squad-v2,\nCOD_MUN,0.000,1 - Completo\n2 - Só Profissionais,0.000
3,Dicionário de Dados,"Na tabela LFCES004, qual a descrição do campo PFPJ_IND",deepset/roberta-base-squad2,CO_UNIDADE,0.084,Indica se é Pessoa Física ou Jurídica,0.000
4,Dicionário de Dados,"Na tabela LFCES004, qual a descrição do campo PFPJ_IND",distilbert-base-cased-distilled-squad,Esse campo fica criptografado,0.003,Indica se é Pessoa Física ou Jurídica,0.000
5,Dicionário de Dados,"Na tabela LFCES004, qual a descrição do campo PFPJ_IND",ktrapeznikov/albert-xlarge-v2-squad-v2,PFPJ_IND = 1,0.001,Indica se é Pessoa Física ou Jurídica,0.000
6,Dicionário de Dados,"Na tabela LFCES035, qual nme do campo responsável por armazenar a Data do Processamento e seu tipo?",deepset/roberta-base-squad2,Senha\n,0.026,DT_PROCESS - DATE,0.000
7,Dicionário de Dados,"Na tabela LFCES035, qual nme do campo responsável por armazenar a Data do Processamento e seu tipo?",distilbert-base-cased-distilled-squad,Acessos,0.000,DT_PROCESS - DATE,0.000
8,Dicionário de Dados,"Na tabela LFCES035, qual nme do campo responsável por armazenar a Data do Processamento e seu tipo?",ktrapeznikov/albert-xlarge-v2-squad-v2,\nNFCES050,0.005,DT_PROCESS - DATE,0.000
9,Doenças Respiratórias,Com quais doenças respiratórias o baqueteamento digital pode estar associado?,deepset/roberta-base-squad2,"cardiovasculares, digestivas e \npulmonares",0.310,"Doenças cardiovasculares, digestivas e pulmonares.",0.777
